In [ ]:
import chess.engine

# Définir le chemin vers l'exécutable de Stockfish
STOCKFISH_PATH = "/usr/games/stockfish" 

# Initialiser le moteur Stockfish
engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)


In [ ]:
def evaluer_coup(echiquier, coup):
    """Analyse un coup avec Stockfish et retourne une évaluation qualitative."""

    # Évaluation avant le coup
    info_avant = engine.analyse(echiquier, chess.engine.Limit(depth=18))
    eval_avant = info_avant["score"].relative.score(mate_score=10000)  # Centipions
    
    # Jouer le coup
    echiquier.push(coup)

    # Évaluation après le coup
    info_apres = engine.analyse(echiquier, chess.engine.Limit(depth=18))
    eval_apres = info_apres["score"].relative.score(mate_score=10000)  # Centipions

    # Annuler le coup pour ne pas modifier l'échiquier
    echiquier.pop()

    # Différence d'évaluation
    variation = eval_apres - eval_avant

    # Déterminer la classification du coup
    if variation >= 300:
        return "!!"  # Coup brillant
    elif 100 <= variation < 300:
        return "!"
    elif -50 <= variation < 0:
        return "?!?"  # Imprécision
    elif -100 <= variation < -50:
        return "?"  # Erreur
    elif variation < -100:
        return "??"  # Grosse erreur
    else:
        return ""  # Coup normal


In [ ]:
from stockfish import Stockfish

stockfish = Stockfish("/usr/games/stockfish")  # Mets le chemin correct ici
stockfish.set_skill_level(10)

print(stockfish.get_best_move())  # Vérifie si Stockfish fonctionne


In [ ]:
import ipycanvas
import ipywidgets as widgets
import os
import chess
import chess.pgn
import chess.engine


# Dimensions de l'échiquier
taille_case = 50
echiquier_size = 8  
chemin_images = "chess_images"

canvas = ipycanvas.Canvas(width=taille_case * echiquier_size, height=taille_case * echiquier_size)
display(canvas)  

# Création du moteur Stockfish
engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")

# Initialisation de l'échiquier
echiquier = chess.Board()

# Charger la partie à partir d'un fichier PGN
def charger_partie_pgn(fichier_pgn):
    with open(fichier_pgn, "r") as pgn:
        partie = chess.pgn.read_game(pgn)
    return list(partie.mainline_moves())

# Dessiner l'échiquier
def dessiner_echiquier():
    for y in range(echiquier_size):
        for x in range(echiquier_size):
            color = "white" if (x + y) % 2 == 0 else "grey"
            canvas.fill_style = color
            canvas.fill_rect(x * taille_case, y * taille_case, taille_case, taille_case)

# Dessiner une pièce sur l'échiquier
def dessiner_piece_image(image_path, x, y):
    if os.path.exists(image_path):  
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')  
        canvas.draw_image(image, x * taille_case, y * taille_case, taille_case, taille_case)

# Mettre à jour l'échiquier
def mettre_a_jour_echiquier():
    canvas.clear()  
    dessiner_echiquier()  
    
    for case in chess.SQUARES:
        piece = echiquier.piece_at(case) 
        if piece:
            x, y = chess.square_file(case), 7 - chess.square_rank(case)  
            image_path = os.path.join(chemin_images, f"{'w' if piece.color else 'b'}{piece.symbol().upper()}.png")
            dessiner_piece_image(image_path, x, y)

# Évaluer un coup avec Stockfish
def evaluer_coup(echiquier, coup):
    """Analyse un coup avec Stockfish et retourne une évaluation qualitative."""
    
    # Évaluation avant le coup
    info_avant = engine.analyse(echiquier, chess.engine.Limit(depth=18))
    eval_avant = info_avant["score"].relative.score(mate_score=10000)  # Centipions
    
    # Jouer le coup
    echiquier.push(coup)

    # Évaluation après le coup
    info_apres = engine.analyse(echiquier, chess.engine.Limit(depth=18))
    eval_apres = info_apres["score"].relative.score(mate_score=10000)  # Centipions

    # Annuler le coup pour ne pas modifier l'échiquier
    echiquier.pop()

    # Différence d'évaluation
    variation = eval_apres - eval_avant

    # Déterminer la classification du coup
    if variation >= 300:
        return "!!"  # Coup brillant
    elif 100 <= variation < 300:
        return "!"  # Coup bon
    elif -50 <= variation < 0:
        return "?!?"  # Imprécision
    elif -100 <= variation < -50:
        return "?"  # Erreur
    elif variation < -100:
        return "??"  # Grosse erreur
    else:
        return ""  # Coup normal

# Appliquer un coup sur l'échiquier
def appliquer_coup(coup):
    global echiquier
    move = chess.Move.from_uci(str(coup))  # Convertir le coup en notation UCI
    if move in echiquier.legal_moves:  
        echiquier.push(move)  # Appliquer le coup
        mettre_a_jour_echiquier()  # Mettre à jour l'affichage
        annotation = evaluer_coup(echiquier, move)  # Évaluer le coup
        afficher_annotation(annotation)

# Afficher l'annotation sous l'échiquier
#def afficher_annotation(annotation):

#   annotation_widget = widgets.Label(f"Évaluation du coup : {annotation}")
#    display(annotation_widget)

def afficher_annotation(annotation):
    
    canvas.fill_style = "black"
    canvas.font = "20px Arial"
    canvas.fill_text(annotation, 10, 10)  # Affiche l'annotation en haut à gauche

# Jouer le prochain coup
coups = []
indice_coup = 0

def jouer_prochain_coup(b):
    global indice_coup
    if indice_coup < len(coups):
        appliquer_coup(coups[indice_coup])
        indice_coup += 1

# Jouer le coup précédent
def jouer_ancien_coup(b):
    global indice_coup
    if indice_coup > 0:  
        echiquier.pop()  
        indice_coup -= 1  
        mettre_a_jour_echiquier() 

# Rejouer la partie à partir d'un fichier PGN
def rejouer_partie(fichier_pgn):
    global echiquier, coups, indice_coup
    echiquier = chess.Board()  # Réinitialiser l'échiquier
    coups = charger_partie_pgn(fichier_pgn)  # Charger les coups
    indice_coup = 0
    mettre_a_jour_echiquier()  # Mettre à jour l'échiquier au début de la partie

    # Boutons pour naviguer entre les coups
    bouton_suivant = widgets.Button(description="Coup suivant")
    bouton_suivant.on_click(jouer_prochain_coup)
    bouton_precedent = widgets.Button(description="Coup précédent")
    bouton_precedent.on_click(jouer_ancien_coup)
    display(bouton_suivant)
    display(bouton_precedent)

# Lancer la partie (mettre ton fichier PGN ici)
rejouer_partie('Loowolf_vs_TelophileGOAT_2025.02.19.pgn')  


In [ ]:
import ipycanvas
import ipywidgets as widgets
import os
import chess
import chess.pgn
from stockfish import Stockfish  # Assurez-vous que Stockfish est installé et fonctionnel

# Dimensions de l'échiquier
taille_case = 50
echiquier_size = 8
chemin_images = "chess_images"
canvas = ipycanvas.Canvas(width=taille_case * echiquier_size, height=taille_case * echiquier_size)
display(canvas)

# Créer un moteur Stockfish
stockfish = Stockfish("/usr/games/stockfish")  # Mettre le chemin d'accès correct pour Stockfish
stockfish.set_skill_level(10)

# Créer l'échiquier de départ
echiquier = chess.Board()

# Charger une partie PGN
def charger_partie_pgn(fichier_pgn):
    with open(fichier_pgn, "r") as pgn:
        partie = chess.pgn.read_game(pgn)
    return list(partie.mainline_moves())

# Dessiner l'échiquier sur le canvas
def dessiner_echiquier():
    for y in range(echiquier_size):
        for x in range(echiquier_size):
            color = "white" if (x + y) % 2 == 0 else "grey"
            canvas.fill_style = color
            canvas.fill_rect(x * taille_case, y * taille_case, taille_case, taille_case)

# Dessiner les pièces sur l'échiquier
def dessiner_piece_image(image_path, x, y):
    if os.path.exists(image_path):
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')
        canvas.draw_image(image, x * taille_case, y * taille_case, taille_case, taille_case)

# Mettre à jour l'échiquier
def mettre_a_jour_echiquier():
    canvas.clear()
    dessiner_echiquier()
    for case in chess.SQUARES:
        piece = echiquier.piece_at(case)
        if piece:
            x, y = chess.square_file(case), 7 - chess.square_rank(case)
            image_path = os.path.join(chemin_images, f"{'w' if piece.color else 'b'}{piece.symbol().upper()}.png")
            dessiner_piece_image(image_path, x, y)

### 
def appliquer_coup(coup):
    global echiquier
    move_chess = None
    move = move_chess 
    move = chess.Move.from_uci(str(coup))  # Convertir le coup en notation UCI
    if move in echiquier.legal_moves:  # Vérifie si le coup est légal
        echiquier.push(move)  # Appliquer le coup
        mettre_a_jour_echiquier()  # Mettre à jour l'affichage
        annotation = evaluer_coup(echiquier, move_chess )  # Évaluer le coup
        afficher_annotation(annotation)  # Afficher l'annotation
    else:
        print(f"Le coup {coup} n'est pas valide ou ne peut pas être joué à ce moment.")
###

# Fonction pour évaluer un coup avec Stockfish
def evaluer_coup(echiquier, coup):
    """Analyse un coup avec Stockfish et retourne une évaluation qualitative."""

    # Évaluation avant le coup
    fen_avant = echiquier.fen()  # Récupérer la position avant le coup
    stockfish.set_fen_position(fen_avant)  # Envoyer la position à Stockfish
    info_avant = stockfish.get_evaluation()  # Obtenir l'évaluation avant le coup
    eval_avant = info_avant['value']  # Récupérer l'évaluation (en centipions)
    
    # Jouer le coup
    echiquier.push(coup)

    # Évaluation après le coup
    fen_apres = echiquier.fen()  # Récupérer la position après le coup
    stockfish.set_fen_position(fen_apres)  # Envoyer la position à Stockfish
    info_apres = stockfish.get_evaluation()  # Obtenir l'évaluation après le coup
    eval_apres = info_apres['value']  # Récupérer l'évaluation (en centipions)

    # Annuler le coup pour ne pas modifier l'échiquier
    echiquier.pop()

    # Différence d'évaluation
    variation = eval_apres - eval_avant

    # Déterminer la classification du coup
    if variation >= 300:
        return "!!"  # Coup brillant
    elif 100 <= variation < 300:
        return "!"
    elif -50 <= variation < 0:
        return "?!?"  # Imprécision
    elif -100 <= variation < -50:
        return "?"  # Erreur
    elif variation < -100:
        return "??"  # Grosse erreur
    else:
        return ""  # Coup normal

# Fonction pour afficher l'annotation sur le canvas
def afficher_annotation(annotation):
    canvas.fill_style = "black"
    canvas.font = "20px Arial"
    canvas.fill_text(f"Annotation: {annotation}", 10, 20)  # Affiche l'annotation en haut à gauche

# Charger une partie et la rejouer
def rejouer_partie(fichier_pgn):
    global echiquier, coups, indice_coup
    echiquier = chess.Board()
    coups = charger_partie_pgn(fichier_pgn)
    indice_coup = 0
    mettre_a_jour_echiquier()

    # Boutons pour avancer et reculer dans les coups
    bouton_suivant = widgets.Button(description="Coup suivant")
    bouton_suivant.on_click(jouer_prochain_coup)
    bouton_precedent = widgets.Button(description="Coup precedent")
    bouton_precedent.on_click(jouer_ancien_coup)
    display(bouton_suivant)
    display(bouton_precedent)

# Jouer le prochain coup
def jouer_prochain_coup(b):
    global indice_coup
    if indice_coup < len(coups):
        appliquer_coup(coups[indice_coup])
        indice_coup += 1

# Jouer l'ancien coup
def jouer_ancien_coup(b):
    global indice_coup
    if indice_coup > 0:
        echiquier.pop()  # Annuler le dernier coup
        indice_coup -= 1
        mettre_a_jour_echiquier()

# Lancer la partie (changez le nom du fichier PGN si nécessaire)
rejouer_partie('Loowolf_vs_TelophileGOAT_2025.02.19.pgn')


Canvas(height=400, width=400)

Button(description='Coup suivant', style=ButtonStyle())

Button(description='Coup precedent', style=ButtonStyle())

AttributeError: 'NoneType' object has no attribute 'from_square'